In [2]:
# !pip install pytube
# !pip install pydub
# !pip install SpeechRecognition
# !pip install imageio==2.4.1      rodar esses comandos pip para instalar as bibliotecas usadas
# !pip install transformers
# !pip install kaggle
# !mkdir audio            #rodar para criar pasta no google colab, no jupyter não é necessário
# !mkdir audio_dividido
# !mkdir Dataset

In [ ]:
# from google.colab import files
# files.upload() 
                            # Não é necessário rodar pois ja baixamos os dados e baixamos o csv para depois dar load
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d najzeko/steam-reviews-2021 -p ./Dataset
# !unzip ./Dataset/steam-reviews-2021.zip
# !rm -r ./Dataset/steam-reviews-2021.zip

In [2]:
import speech_recognition as sr
import moviepy.editor as mp
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import sys
import pydub
import re
import os

from IPython.display import clear_output
from IPython.core.display import HTML
from pytube import YouTube
from pydub.utils import make_chunks
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from nltk.corpus import wordnet
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

In [2]:
# from google.colab import drive

# drive.mount('/content/drive')

Professor, vou colocar comentários para que o senhor rode a célula

In [3]:
nltk.download('stopwords') # rodar

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dayvison\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def download_audio(link, path_download): # rodar
    
    yt = YouTube(link)
    ys = yt.streams.filter(only_audio=True).first().download(path_download)
    
    for file in os.listdir(path_download):         
        if re.search('mp4', file):                                    
            mp4_path = os.path.join(path_download , file)   
            name_video = os.path.splitext(file)[0]
            mp3_path = os.path.join(path_download, os.path.splitext(file)[0]+'.mp3')
            
            new_file = mp.AudioFileClip(mp4_path) 
            new_file.write_audiofile(mp3_path) 
            os.remove(mp4_path)                    
    
#     print("Download Completo")
    return name_video

In [ ]:
name_audio = download_audio('https://www.youtube.com/watch?v=FUcSmOBAJHA', './audio/')

[MoviePy] Writing audio in ./audio/Batman Arkham Knight FINAL 100  COMPLETO PROTOCOLO KNIGHTFALL EM PORTUGUÊS.mp3


100%|██████████| 8232/8232 [00:13<00:00, 620.37it/s]

[MoviePy] Done.


In [5]:
def convert_and_divide(name_audio, gradio=False, tam=30000): # rodar
    
    if gradio:
        
        audio_mp3 = name_audio + '.mp3'
        audio_wav = name_audio + '.wav'
    
    else:
        audio_mp3 = './audio/' + name_audio + '.mp3'
        audio_wav = './audio/' + name_audio + '.wav'

    sound = pydub.AudioSegment.from_mp3(audio_mp3)
    sound.export(audio_wav, format='wav')

    audio = pydub.AudioSegment.from_file(audio_wav, 'wav')
    tamanho = tam

    partes = make_chunks(audio, tamanho) 
    partes_audio = []

    for i, parte in enumerate(partes):

        parte_name = 'audio_parte{0}.wav'.format(i)
        partes_audio.append('./audio_dividido/' + parte_name)
        parte.export('./audio_dividido/' + parte_name, format='wav')
        
    return partes_audio


In [6]:
def transcreve_audio(nome_audio): # rodar
    r = sr.Recognizer()
    
    with sr.AudioFile(nome_audio) as source:
        audio = r.record(source)      
        try:
            texto = r.recognize_google(audio, language='pt-BR')
        except sr.UnknownValueError:
            print('Fala direito')
            texto = 'vazio '
        except sr.RequestError as e:
            print('Erro ao solicitar resultados do serviço Google Speech Recognition; {0}'.format(e))
            texto = 'vazio '
            
    return texto

In [ ]:
texto = ''
frases = []
partes_audio = convert_and_divide(name_audio, 30000)

for parte in partes_audio:
    transcrito = transcreve_audio(parte)
    frases.append(transcrito)
    texto = texto + ' ' + transcrito

### Modelo do Zero

#### Pré-Processamento

In [7]:
# frases_audio = pd.DataFrame(data={'Frases':frases, 'Sentimento':[0 for i in range(len(frases))]})
frases_audio = pd.read_csv('frases_audio.csv')

In [ ]:
df = pd.read_csv('./Dataset/steam_reviews.csv', nrows=10000000)

In [ ]:
df_ = df[(df['language'] == 'portuguese') | (df['language'] == 'brazilian')]

In [ ]:
df_filter = df_[['review','recommended']]

In [ ]:
df_filter['recommended'] = df_filter['recommended'].apply(lambda x: 1 if x else 0)

In [ ]:
df_filter['recommended'].value_counts()

In [ ]:
under_sampling = RandomUnderSampler(random_state=42)
X = df_filter['review'].values.reshape(-1,1)
y = df_filter['recommended'].values.reshape(-1,1)
X_resampled, y_resampled = under_sampling.fit_resample(X, y)

In [ ]:
df_resampled = pd.DataFrame(data={'Frases':X_resampled.ravel(), 'Sentiment': y_resampled})

In [ ]:
frases_remove = []

for frase in df_resampled['Frases']:
  try:

    if len(frase) <= 2:
      frases_remove.append(frase)

  except:
      frases_remove.append(frase)

In [ ]:
frases_remove = np.unique(frases_remove)

In [ ]:
index = []

for frase in frases_remove:
  filtro = df_resampled[df_resampled['Frases'] == frase].index.to_list()
  index.extend(filtro)

In [ ]:
df_resampled = df_resampled.drop(index)

In [ ]:
df_resampled = df_resampled.dropna(subset=['Frases']).reset_index().drop(columns=['index'])

In [ ]:
del df

In [ ]:
# df_resampled.to_csv('reviews_filtro.csv')

In [7]:
df_resampled = pd.read_csv('reviews_filtro.csv') #rodar

In [8]:
stop_words = nltk.corpus.stopwords.words('portuguese') #rodar

In [9]:
def get_only_chars(line): # rodar

    clean_line = ""
    
    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") 
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line)
    
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]

    return clean_line

def get_vocab(corpus):
    vocab = []
    
    for frase in corpus:
        words = frase.split()
        
        for word in words:
            vocab.append(word)
  
    return vocab

In [10]:
#rodar

# frases_audio['Frases'] = frases_audio['Frases'].apply(lambda x: get_only_chars(x))
# frases_audio['Frases'] = frases_audio['Frases'].apply(lambda x: " ".join([y for y in x.split() if y not in stop_words]  ))

df_resampled['Frases'] = df_resampled['Frases'].apply(lambda x: get_only_chars(x))
df_resampled['Frases'] = df_resampled['Frases'].apply(lambda x: " ".join([y for y in x.split() if y not in stop_words]  ))

In [11]:
#rodar
X_train, X_test, y_train, y_test = train_test_split(df_resampled['Frases'], df_resampled['Sentiment'], random_state=42, test_size=0.3)

In [ ]:
vectorize_layer = keras.layers.TextVectorization(max_tokens=15000, output_mode='int', output_sequence_length=len(max(X_train)))
vocab = get_vocab(X_train)
vectorize_layer.adapt(np.unique(vocab)) 

model = keras.models.Sequential([
    vectorize_layer,
    keras.layers.Embedding(input_dim=len(vectorize_layer.get_vocabulary()), output_dim=32),
    keras.layers.LSTM(16, activation='relu',kernel_regularizer=keras.regularizers.L2(0.01), return_sequences=True),
    keras.layers.LSTM(8, activation='relu', kernel_regularizer=keras.regularizers.L2(0.01)),
    
    keras.layers.Dense(1, activation='sigmoid')
])
  
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)
y_pred = np.where(y_pred >= 0.5, 1, 0)

print(" ")
print(classification_report(y_test, y_pred) )
print(" ")

plt.figure(figsize=(12,6))
plt.title('Loss and Acc')
plt.grid()
plt.plot(history.history['loss'], color='darkred', label='Loss')
plt.plot(history.history['val_loss'], color='green', label='Val_Loss')
plt.legend()

Salvando o modelo

In [12]:
#model.save('model') # rodar
loaded_model = keras.models.load_model('model', custom_objects={'LSTMCell': keras.layers.LSTM})

y_preds = loaded_model.predict(X_test)
y_preds = np.where(y_preds >= 0.5, 1, 0)

In [13]:
print(classification_report(y_test, y_preds)) # rodar

              precision    recall  f1-score   support

           0       0.87      0.47      0.61      2453
           1       0.64      0.93      0.76      2450

    accuracy                           0.70      4903
   macro avg       0.76      0.70      0.69      4903
weighted avg       0.76      0.70      0.69      4903



In [ ]:
frases_audio['Sentimento'] = loaded_model.predict(frases_audio['Frases'])

1/1 [==============================] - 0s 92ms/step


In [ ]:
frases_audio['Sentimento'] = frases_audio['Sentimento'].apply(lambda x: 1 if x >= 0.5 else 0)

### Modelo Pré-Treinado - Bônus

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification # rodar
from torch.nn.functional import softmax
  
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model_ = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def map_values(value, min = 1, max = 5, feature_range = (-1, 1)):

  # Map to feature range
    mapped = (((value - min) * (feature_range[1] - feature_range[0])) / (max - min)) + feature_range[0]

    return mapped

def predict_polarity(sentence, model_, tokenizer):
    # Generate tokens
    
    tokens = tokenizer.encode(sentence, return_tensors='pt', truncation=True)

  # Predict logits and probability
    logits = model_(tokens)[0][0]
    probas = softmax(logits, dim=0).detach().numpy()

  # Calculate center of mass (1*p1 + ... + 5*p5)/sum(p)
    center_of_mass = np.average(np.arange(1, len(probas)+1), weights = probas)

  # Map value to (-1, 1)
    return map_values(center_of_mass)

In [ ]:
polarity = []

for frase in frases_audio['Frases']:
    polarity.append(predict_polarity(frase, model_, tokenizer))

In [ ]:
frases_audio['Sentimento_bert'] = polarity

In [ ]:
frases_audio = frases_audio.drop(columns=['Unnamed: 0'])

In [ ]:
tempo_init = 0
frases_audio['Tempo'] = [(tempo_init + (i*30000))/60000 for i in range(len(partes_audio))]

In [ ]:
frases_audio

In [ ]:
%matplotlib inline

In [ ]:
x = []
y = []

tempos = frases_audio['Tempo'].values
sentimento = frases_audio['Sentimento_treinado'].values

for i in range(len(tempos)):
    x.append(tempos[i])
    y.append(sentimento[i])
    
    plt.figure(figsize=(10,4))
    plt.title('Sentimento ao longo do tempo')
    plt.grid()
    plt.xlim(0, tempos[-1])
    plt.ylim(-1, 1)
    
    plt.plot(x, y, f'-{"r" if y[i] <= 0 else "b"}o')
    plt.pause(0.5)
#     plt.pause(30000/1000)
    clear_output(wait = True)



In [ ]:
HTML('<iframe id="ytplayer" type="text/html" width="600" height="400" \ src="https://www.youtube.com/embed/XFsF-BTb24g" frameborder="2"/>')

### Gradio

In [15]:
# !pip install gradio

Link de exemplo de vídeo: https://www.youtube.com/watch?v=uPeBB3KoIu0

In [15]:
import gradio as gr

def read_input(link): # rodar
    
    path = './audio/'
    name_video = download_audio(link, path)
    path_name = path + name_video
    
    tam = 30000
    partes_audio = convert_and_divide(path_name, gradio=True, tam=tam)
    
    frases = []
    
    for parte in partes_audio:
        transcrito = transcreve_audio(parte)
        frases.append(transcrito)
    
    frases_audio = pd.DataFrame(data={'Frases':frases})
    frases_audio['Frases'] = frases_audio['Frases'].apply(lambda x: get_only_chars(x))
    frases_audio['Frases'] = frases_audio['Frases'].apply(lambda x: " ".join([y for y in x.split() if y not in stop_words]  ))
    
    df = pd.DataFrame(columns=["Text","Sentimento_modelo_keras", "Sentimento_modelo_bert", "Time"])
    
    tempo_init = 0
    
    y_pred_keras = loaded_model.predict(frases_audio['Frases'])
    y_pred_keras = np.where(y_pred_keras >= 0.5, 1, 0)
    
    for i in range(len(y_pred_keras)):
        frase = frases_audio['Frases'].iloc[i]
        
        y_pred_bert = predict_polarity(frase, model_, tokenizer)
        df.loc[len(df)] = [frase, str(y_pred_keras[i][0]), str(y_pred_bert), (tempo_init + (i*tam))/60000]
    
    
    tempos = df['Time'].values
    sentimento = df['Sentimento_modelo_keras'].values
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.grid()
    ax.plot(tempos, sentimento, '-bo')  
    
      
     
    return {output_Dataframe : df , output_plot : fig}


with gr.Blocks() as demo:
    with gr.Tab("Link do Vídeo"):
        with gr.Row():
            with gr.Column(min_width="400"):
            
                link = gr.Textbox(label="Digite o link")
                download = gr.Button("Enviar")
                        
              
    with gr.Tab("Resultados"):
        with gr.Row():
            with gr.Column(min_width="400"):
                
                output_Dataframe = gr.Dataframe(
                    headers=["Text","Sentimento_modelo_keras", "Sentimento_modelo_bert","Time"],
                    datatype=["str","str", "str","str"]
                    )
                
                output_plot = gr.Plot(label='Plot')
                         
    download.click(read_input, [link], [output_plot, output_Dataframe])  
    
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://10a4a7d21fc6eb35.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


MoviePy - Writing audio in ./audio/É o FIM do BATMAN ARKHAM.mp3


MoviePy - Done.
result2:
{   'alternative': [   {   'confidence': 0.91770357,
                           'transcript': 'desde 2015 com final do Batman '
                                         'Arkham Knight nunca mais tivemos o '
                                         'jogo próprio do homem morcego isso '
                                         'de certa forma é preocupante para os '
                                         'fãs de Batman Mas será que a gente '
                                         'vai ter um novo jogo do Batman no '
                                         'futuro não querendo dar spoiler '
                                         'daqui mas afinal do Bat marca não '
                                         'terminou de uma forma definitiva + 5 '
                                         'e aberto Ou seja terá margem para '
                                         'rock Terry continuar a Frankie arca '
                                         'sem precisar inv

result2:
{   'alternative': [   {   'confidence': 0.93103141,
                           'transcript': 'aí vem aqui então apesar do origem '
                                         'ter sido um bom jogo na minha '
                                         'opinião de gente que não gostou Mas '
                                         'eu particularmente gostei o último '
                                         'game lançado pela Warner Warner Bros '
                                         'foi agora night esse jogo aí City '
                                         'jogo não curte de forma alguma não '
                                         'me agradou Então para mim é muito '
                                         'arriscado pois o futuro da Frankie '
                                         'arca na mão de outra empresa '
                                         'anunciar bactéria Porque '
                                         'infelizmente é aqui falando até pela '
        

result2:
{   'alternative': [   {   'confidence': 0.93498647,
                           'transcript': 'são Nesse contexto mas agora com cor '
                                         'com quatro jogo do Batman Será que '
                                         'não chegaria no nível de saturação '
                                         'de repetição grande daí que faz '
                                         'imaginar uma continuação assim no '
                                         'ano seguinte ou depois de dois anos '
                                         'eu posso ficar aqui algum exemplo de '
                                         'game que tem um furo no passado e '
                                         'agora quero uma reflexão para a '
                                         'maior parte dos jogadores que a '
                                         'franquia Far Cry então Na minha '
                                         'opinião eu percebi que a bactéria e 